In [1]:
import umap.umap_ as umap


# Create an instance of the UMAP model
reducer = umap.UMAP(random_state=42, n_neighbors=15, min_dist=0.1,  n_components=2, metric='euclidean')

# Fit the model to your TF-IDF data and transform the data
reduced_data_umap = reducer.fit_transform(original_data)

Model accuracy: 31.76%


In [ ]:
import umap.umap_ as umap


# Create an instance of the UMAP model
reducer = umap.UMAP(random_state=42, n_neighbors=15, min_dist=0.1,  n_components=2, metric='euclidean')

# Fit the model to your TF-IDF data and transform the data
reduced_data_umap = reducer.fit_transform(original_data)

### XGBoost
According to [XGBoost: Introduction to XGBoost Algorithm in Machine Learning](https://www.analyticsvidhya.com/blog/2018/09/an-end-to-end-guide-to-understand-the-math-behind-xgboost), it is a machine learning algorithm, utilizes the gradient boosting frameowrk, and would combine multiple individual models, often decision trees, to build an ensemble learning model.

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb

def handle_xgboost(X, y):
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
	X_train_csr = csr_matrix(X_train)
	X_test_csr = csr_matrix(X_test)

	dtrain = xgb.DMatrix(X_train_csr, label=y_train)
	dtest = xgb.DMatrix(X_test_csr, label=y_test)

	# Define your model parameters
	params = {
	    'objective': 'multi:softmax',  # Use 'binary:logistic' if you have two classes
	    'num_class': len(np.unique(y)),  # Needed for multi-class classification
			'tree_method': 'hist',  # Faster histogram optimized algorithm
	}
	evals = [(dtest, 'eval')]
	num_boost_round = 1000  # Set higher for early stopping
	early_stopping_rounds = 10

	# Train the model
	bst = xgb.train(params, dtrain, num_boost_round, evals=evals, early_stopping_rounds=early_stopping_rounds)

	# Make predictions
	predictions = bst.predict(dtest)
	return y_test, predictions

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def print_precision(y_test, p):
	# Accuracy
	accuracy = accuracy_score(y_test, p)
	print(f"Accuracy: {accuracy}")

	# Precision
	precision = precision_score(y_test, p, average='macro')
	print(f"Precision: {precision}")

	# Recall
	recall = recall_score(y_test, p, average='macro')
	print(f"Recall: {recall}")

	# F1 Score
	f1 = f1_score(y_test, p, average='macro')
	print(f"F1 Score: {f1}")




During my processing period, the key difference is its effectiveness. It is too slow to run on the whole dataset, but also the parameter tuning. With default parameter and a tfidf dataset, the XGBoost model was running 74 minutes in my computer, but still not end. 

Think of previous assignment, dimension reduction maybe a good way. So we tried the PCA analysis. After running the model with a 100 features PCA result, we get the precision: 

In [ ]:
y_pca, p_pca = handle_xgboost(reduced_data, y)
print_precision(y_pca, p_pca)

It seems not so well but can be updated, and it takes 3.5 minutes. But I hope to reduce more and make it within 1 minutes.

So I tried to reduce the train model size. With a 0.98 test size, which means only use 0.02, nearly 4,000 rows to train the model, the time is only 14s. However, the result is 

```
Accuracy: 0.4427404705435455
Precision: 0.41335755375629696
Recall: 0.25124844089816784
F1 Score: 0.2880839218375771
```

It is not so good. I think the amount is too less and make the whole model underfit. 


Then we tried the UMAP to reduce the whole dataset to 2d. The efficiency improves greatly, we only need 17s to run the whole dataset. However, the precision is not so good: 

In [ ]:

y_umap, p_umap = handle_xgboost(reduced_data_umap, y)
print_precision(y_umap, p_umap)


It is worse than only use 2% data. Tthe UMAP ignores too many features.

After studying, I found that the stratify sample may be a good way to find a good parameter. And for avoiding the computing burdern of all parameter combination, we decided only consider this 2 kinds of parameters: `max_depth` and `reg_lambda`.

```
param_grid = {
    'max_depth': [3, 5, 7],
    'reg_lambda': [1, 10]
}
```


In [ ]:
# from sklearn.model_selection import train_test_split, GridSearchCV
# from xgboost import XGBClassifier

# # Assuming X, y represent your full dataset
# # Select a random subset
# X_subset, X_val, y_subset, y_val = train_test_split(reduced_data, y, test_size=0.9, stratify=y, random_state=42)

# # Define your model and parameter grid
# model = XGBClassifier()
# param_grid = {
#     'max_depth': [3, 5, 7],
#     'reg_lambda': [1, 10],
# }

# # Perform grid search on the subset
# grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
# grid_search.fit(X_subset, y_subset)

# # Sorting the results by the mean test score in descending order
# results_df = pd.DataFrame(grid_search.cv_results_)
# sorted_results_df = results_df.sort_values(by='rank_test_score')

# print(sorted_results_df)


After 1 hour running, We found max_depth =7 and reg_lambda = 10 is the best, but the time is so long that I cannot run other parameters. We would run XGBoost model with this parameter.

In [ ]:
X = reduced_data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_csr = csr_matrix(X_train)
X_test_csr = csr_matrix(X_test)

dtrain = xgb.DMatrix(X_train_csr, label=y_train)
dtest = xgb.DMatrix(X_test_csr, label=y_test)

# Define your model parameters
params = {
    'objective': 'multi:softprob',  # Use 'binary:logistic' if you have two classes
    'num_class': len(np.unique(y)),  # Needed for multi-class classification
		'tree_method': 'hist',  # Faster histogram optimized algorithm
		'max_depth': 7,
		'reg_lambda': 10
}
evals = [(dtest, 'eval')]
num_boost_round = 1000  # Set higher for early stopping
early_stopping_rounds = 10

# Train the model
bst = xgb.train(params, dtrain, num_boost_round, evals=evals, early_stopping_rounds=early_stopping_rounds)

# Make predictions
predictions = bst.predict(dtest)


With above XGBoost model, we get the result, confusion matrix and ROC curves.

In [ ]:
import numpy as np

# Convert probability predictions to predicted class labels
predicted_labels = np.argmax(predictions, axis=1)
print_precision(y_test, predicted_labels)
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming y_test contains your true labels
true_labels = y_test  # Replace y_test with your actual true labels
conf_matrix = confusion_matrix(true_labels, predicted_labels)  # Use predictions_binary or the equivalent for your case

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=['Class1', 'Class2'], yticklabels=['Class1', 'Class2'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
print(conf_matrix)

# ROC Curve
from sklearn.preprocessing import label_binarize

# Assuming you have n_classes classes
n_classes = len(np.unique(true_labels))  # Make sure true_labels is your array of true labels

true_labels_binarized = label_binarize(true_labels, classes=range(n_classes))
from sklearn.metrics import roc_curve, auc

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(true_labels_binarized[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
plt.figure(figsize=(7, 7))
colors = ['blue', 'red', 'green', 'purple', 'orange']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC')
plt.legend(loc="lower right")
plt.show()

It seems work well in ROC curve and confusion matrix, but the accuracy, precision, recall, and f1score are not so good. What I get is the model works well in some classes, but not so good in the precision part.